# HDF to Geotif

In [9]:
from osgeo import gdal
import os


In [7]:
## List input raster files
os.chdir(r'C:\Users\admin\Downloads\nightdata')
rasterFiles = os.listdir(os.getcwd())
print(rasterFiles[:10])


['VNP46A4.A2020001.h00v01.001.2021125044916.h5', 'VNP46A4.A2020001.h00v02.001.2021125090221.h5', 'VNP46A4.A2020001.h00v03.001.2021118145651.h5', 'VNP46A4.A2020001.h00v06.001.2021118145653.h5', 'VNP46A4.A2020001.h00v09.001.2021125122627.h5', 'VNP46A4.A2020001.h00v10.001.2021125013226.h5', 'VNP46A4.A2020001.h00v11.001.2021125122620.h5', 'VNP46A4.A2020001.h00v13.001.2021119101536.h5', 'VNP46A4.A2020001.h00v14.001.2021119101505.h5', 'VNP46A4.A2020001.h00v15.001.2021121003934.h5']


In [5]:
#Get File Name Prefix (without format)
rasterFilePre = rasterFiles[0][:-3]
print(rasterFilePre)

fileExtension = "_BBOX.tif"

VNP46A4.A2020001.h00v01.001.2021125044916


In [ ]:
for file in rasterFiles:
    rasterFilePre = file[:-3]
    #print(rasterFilePre)

    fileExtension = "_BBOX.tif"
    ## Open HDF file
    hdflayer = gdal.Open(file, gdal.GA_ReadOnly)

    #print (hdflayer.GetSubDatasets())

    # Open raster layer
    #hdflayer.GetSubDatasets()[0][0] - for first layer
    #hdflayer.GetSubDatasets()[1][0] - for second layer ...etc
    subhdflayer = hdflayer.GetSubDatasets()[0][0]
    rlayer = gdal.Open(subhdflayer, gdal.GA_ReadOnly)
    #outputName = rlayer.GetMetadata_Dict()['long_name']

    #Subset the Long Name
    outputName = subhdflayer[92:]

    outputNameNoSpace = outputName.strip().replace(" ","_").replace("/","_")
    outputNameFinal = outputNameNoSpace + rasterFilePre + fileExtension
    #print(outputNameFinal)

    outputFolder = r"C:\Users\admin\Downloads\nightdata_output_folder"

    outputRaster = outputFolder + "\\" + outputNameFinal

    #collect bounding box coordinates
    HorizontalTileNumber = int(rlayer.GetMetadata_Dict()["HorizontalTileNumber"])
    VerticalTileNumber = int(rlayer.GetMetadata_Dict()["VerticalTileNumber"])
        
    WestBoundCoord = (10*HorizontalTileNumber) - 180
    NorthBoundCoord = 90-(10*VerticalTileNumber)
    EastBoundCoord = WestBoundCoord + 10
    SouthBoundCoord = NorthBoundCoord - 10

    EPSG = "-a_srs EPSG:4326" #WGS84

    translateOptionText = EPSG + " -a_ullr " + str(WestBoundCoord) + " " + str(NorthBoundCoord) + " " + str(EastBoundCoord) + " " + str(SouthBoundCoord)

    translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
    gdal.Translate(outputRaster,rlayer, options=translateoptions)

    hdflayer = None
    rlayer = None

    #Display image in QGIS (run it within QGIS python Console) - remove comment to display
    #iface.addRasterLayer(outputRaster, outputNameFinal)



In [ ]:
#delete xml files that are generated after opening the hdf files

post_file_list = os.listdir(os.getcwd())
for item in post_file_list:
    if item.endswith(".xml"):
        os.remove(os.path.join(os.getcwd(), item))

In [11]:
#Merge all the data
#output files list
os.chdir(r'C:\Users\admin\Downloads\nightdata_output_folder')
rasterOutputFiles = os.listdir(os.getcwd())
print(rasterOutputFiles[:10])

#Merge the data
g = gdal.Warp("output_nodata_2.tif", 
                rasterOutputFiles, 
                format= "GTiff",
                # format = 'MEM',
                srcNodata = 65535,
                dstNodata = 65535,
                options=["COMPRESS=DEFLATE", "TILED=YES"])




g = None # Close file and flush to disk




['elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v01.001.2021125044916_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v02.001.2021125090221_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v03.001.2021118145651_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v06.001.2021118145653_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v09.001.2021125122627_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v10.001.2021125013226_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v11.001.2021125122620_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v13.001.2021119101536_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v14.001.2021119101505_BBOX.tif', 'elds_AllAngle_Composite_Snow_CoveredVNP46A4.A2020001.h00v15.001.2021121003934_BBOX.tif']


In [ ]:
# read in data from first band of input raster
band1 = g.GetRasterBand(1)
rows = g.RasterYSize
cols = g.RasterXSize
vals = band1.ReadAsArray(0, 0, cols, rows)

# create single-band float32 output raster
driver = g.GetDriver()
out_data = driver.Create(raster_output, cols, rows, 1, GDT_Float32)
if out_data is None:
    print 'Could not create output file %s' % raster_output
    return None